String zu Embeddings:

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings
import pyperclip

embeddings = OpenAIEmbeddings()
vec = embeddings.embed_query("Erzähl mir alles was du über Padme weißt")
vec_padme = ""
def digits_to_string(digits):
    # Transform the list of digits into a string of numbers separated by commas
    return ', '.join(str(digit) for digit in digits)
vec_padme = digits_to_string(vec)

pyperclip.copy(vec_padme) # speichern in Zwischenablage

In [ ]:
from langchain_community.llms import Ollama
model = Ollama(model="llama3")

model.invoke("Tell me a joke")

In [2]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai.embeddings import OpenAIEmbeddings
from langsmith import Client
import os 
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI 
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate 
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from pinecone import Pinecone, ServerlessSpec


from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from operator import itemgetter

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Initialize vectorstore
index_pinecone = "llm-km"
embeddings = OpenAIEmbeddings()
vectorstore = PineconeVectorStore(index_name=index_pinecone, embedding=embeddings)

# tracing mit Langsmith from Langchain
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "RAG-project"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
client = Client()

embeddings = OpenAIEmbeddings()

# konvertiert AI Message zu einem String
parser = StrOutputParser() 

# Embedding der Wissenschunks

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo", temperature=0)
# template prompt
rag_template = """
    Beantworte die Frage basierend auf dem gegebenen Kontext: {context}
    
    Wenn es Widersprüche gibt, gib zusätzlich zu der Antwort die Widersprüche seperat aus.      
    Wenn das Beantworten der Frage nicht möglich ist durch den gegebenen Kontext oder kein Kontext gegeben wurde, antworte IMMER "Ich weiß es nicht". 
    """

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", rag_template),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ])        

retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 10, "score_threshold": 0.9})

rewriting_template = """
    Mit der gegebenen Konversation, generiere eine Suchabfrage zum Nachschlagen, um Informationen zu erhalten, die für die Konversation relevant sind. Gib nur die wirkliche Suchabfrage aus.
    """

rewriting_prompt = ChatPromptTemplate.from_messages([MessagesPlaceholder(variable_name = "chat_history"), 
                                                   ("user", "{input}"),
                                                   ("system", rewriting_template)
                                                   ])       


# compose page content of Documents to one string
def format_docs(docs):
    global relevant_docs
    relevant_docs = docs
    return "\n\n".join([d.page_content for d in docs])

# RunnableParallel so both values of the dictionary get parallel excecuted, itemgetter to get the values of keys from the input dictionary
retrieval = RunnableParallel({"context": itemgetter("input") | retriever | format_docs, "input": itemgetter("input"), "chat_history": itemgetter("chat_history")})

# rewrite the query given the chat history
rewriting = RunnableParallel({"input": rewriting_prompt | model | parser, "chat_history": itemgetter("chat_history")})

# do nothing 
no_rewriting = RunnablePassthrough()

def route(input_dict):
    if len(input_dict["chat_history"])!=0:
        return rewriting
    else:
        return no_rewriting


rag_chain = route | retrieval | rag_prompt | model | parser

rag_chain.invoke({
    "chat_history": [],
    "input": "Wie heißen ihre Kinder?"
    })




/Users/maxschneider/projects/llmforkm2/.venv/lib/python3.12/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.9
  warnings.warn(


'Ich weiß es nicht.'